GitHub  
https://github.com/shonenkov/CLIP-ODS  
論文  
https://arxiv.org/abs/2112.14757   
  
<a href="https://colab.research.google.com/github/kaz12tech/ai_demos/blob/master/Clip_Object_DetectionandSegmentation.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Colaboratory環境の確認
Google ColaboratoryにインストールされているGPUドライバーは定期的にアップデートされます。  
GPUドライバーにあったPytorchをインストールするためGPUドライバーのバージョンを確認します。

In [ ]:
import subprocess

CUDA_version = [s for s in subprocess.check_output(["nvcc", "--version"]).decode("UTF-8").split(", ") if s.startswith("release")][0].split(" ")[-1]

if CUDA_version == "10.0":
    torch_version_suffix = "+cu100"
elif CUDA_version == "10.1":
    torch_version_suffix = "+cu101"
elif CUDA_version == "10.2":
    torch_version_suffix = ""
elif "11." in CUDA_version:
    torch_version_suffix = "+cu110"
else:
  torch_version_suffix = "+cpu"

print("CUDA version:", CUDA_version, " Suffix:", torch_version_suffix)

# Pytorchのインストール
Pytorch1.7.1をインストールします。

In [ ]:
!pip install --upgrade pip
!pip uninstall torch -y
!pip install torch==1.7.1{torch_version_suffix} torchvision==0.8.2{torch_version_suffix} torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

# ライブラリのインストール
その他CLIP-ODSに必要なライブラリをインストールします。

In [ ]:
!pip install ftfy regex
!pip install clip-ods==0.0.1rc2

# ライブラリのインポート
先ほどインストールしたライブラリをインポートします

In [ ]:
import torch
import gdown
from google.colab import files
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
from clip_ods import clip, CLIPDetectorV1

# モデルのロード
インストールしたclip_odsライブラリを使って事前学習済みモデルをロードします。

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device = torch.device(device)
base_model = 'RN50x4' #@param ["ViT-B/32","RN50","RN101","RN50x4"]
model, preprocess = clip.load("RN50x4", device=device)
clip_detector = CLIPDetectorV1(model, preprocess, device)

# 画像準備
物体検出を行う画像をアップロードしてください。  
sampleを選択した場合CLIP-odsライブラリ提供元が配布するサンプル画像を使用します。
  
本記事では以下のロイヤリティフリー画像を使用します。
https://pixabay.com/ja/photos/%e5%ad%a6%e7%94%9f-%e3%82%bf%e3%82%a4%e3%83%94%e3%83%b3%e3%82%b0-%e3%82%ad%e3%83%bc%e3%83%9c%e3%83%bc%e3%83%89-849825/  

In [ ]:
%cd /content/
!mkdir images
%cd /content/images

image_type ='upload' #@param ['sample', 'upload']
# sample選択時
if image_type == "sample":
  for google_drive_file_id in ['1nMPyWquE7U7_fuh0Rk4ZGgeWAtCFEqi8','1bsaZ1FSAfMByWeT4Ftr5_J5YaWARUu-x', '1lwhhDDBGztqxW4AVYqjjwKMpda19Vpbu']:
    gdown.download(f'https://drive.google.com/uc?id={google_drive_file_id}', './', quiet=True)
  image_path = "/content/images/example11.jpg"
# upload選択時
else:
  uploaded = files.upload()
  uploaded = list(uploaded.keys())
  file_name = uploaded[0]
  image_path = "/content/images/" + file_name

# 画像の表示
image = Image.open(image_path).convert("RGB")
plt.figure(figsize=(6, 6))
plt.imshow(image)

# 学習
準備した画像を使って物体検出を行います。
始めに画像から特徴を学習します

In [ ]:
%%time

coords, masks = clip_detector.get_coords_and_masks(Image.open(image_path))
anchor_features = clip_detector.get_anchor_features(Image.open(image_path), coords)

# セマンティックセグメンテーション
学習した特徴を使用しセマンティックセグメンテーションを行います。  
textで指定した物体を画像中から検出します。  

In [ ]:
%%time

text = 'watch' #@param {type:"string"}

result = clip_detector.detect_by_text(
    texts=[text],
    img=Image.open(image_path),
    coords=coords, # detect coords
    masks=masks, # Segmentation mask
    anchor_features=anchor_features,
    skip_box_thr=0.7
)

img = Image.open(image_path)
colour = (0,255,0)

img = clip_detector.draw(
    img, 
    result,
    label=text,
    colour=colour,
    font_colour=colour,
    font_scale=0.5, 
    font_thickness=1,
)

plt.figure(num=None, figsize=(8, 8), dpi=120, facecolor='w', edgecolor='k')
plt.imshow(img)

# 物体検出
セグメンテーションマスクをOFFにして物体検出のみ表示します。

In [ ]:
%%time

text = 'watch' #@param {type:"string"}

result = clip_detector.detect_by_text(
    texts=[text],
    img=Image.open(image_path),
    coords=coords,
    #masks=masks, # Segmentation maskなし
    anchor_features=anchor_features,
    skip_box_thr=0.7
)

img = Image.open(image_path)
colour = (0,255,0)

img = clip_detector.draw(
    img, 
    result,
    label=text,
    colour=colour,
    font_colour=colour,
    font_scale=0.5, 
    font_thickness=1,
)

plt.figure(num=None, figsize=(8, 8), dpi=120, facecolor='w', edgecolor='k')
plt.imshow(img);